In [23]:
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import numpy as np

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [24]:
def read_file(path):
  # Read file into an array of strings
  lines = []
  with open(path, encoding="utf8") as file:
      lines = file.readlines()
  # Remove newline characters from each line
  lines = [line.strip() for line in lines]
  return lines

In [25]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = "/content/drive/My Drive/The Tale of Genji (Royall Tyler Translation)(Cleaned - With Titles).txt"

lines = read_file(data_dir)
lines[:5]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['1',
 '',
 'KIRITSUBO',
 '',
 "In a certain reign (whose can it have been?) someone of no very great rank, among all His Majesty's Consorts and Intimates, enjoyed exceptional favor. Those others who had always assumed that pride of place was properly theirs despised her as a dreadful woman, while the lesser Intimates were unhappier still. The way she waited on him day after day only stirred up feeling against her, and perhaps this growing burden of resentment was what affected her health and obliged her often to withdraw in misery to her home; but His Majesty, who could less and less do without her, ignored his critics until his behavior seemed bound to be the talk of all."]

In [26]:
def split_into_chapters(lines):
    chapters = []
    current_chapter = []
    skip_next_line = False

    for line in lines:
        if line.strip().isdigit():
            # Start of a new chapter
            if current_chapter:
                chapters.append(current_chapter)
                current_chapter = []
            skip_next_line = True  # Skip the next line (chapter title)
        elif skip_next_line:
            # Skip this line (chapter title) and continue to the next line
            skip_next_line = False
            continue
        else:
            current_chapter.append(line)

    if current_chapter:
        chapters.append(current_chapter)

    return chapters

# Read file and split into chapters
lines = read_file(data_dir)
chapters = split_into_chapters(lines)

# Example: print the first few lines of the first chapter's content
print(chapters[0][:5])


['KIRITSUBO', '', "In a certain reign (whose can it have been?) someone of no very great rank, among all His Majesty's Consorts and Intimates, enjoyed exceptional favor. Those others who had always assumed that pride of place was properly theirs despised her as a dreadful woman, while the lesser Intimates were unhappier still. The way she waited on him day after day only stirred up feeling against her, and perhaps this growing burden of resentment was what affected her health and obliged her often to withdraw in misery to her home; but His Majesty, who could less and less do without her, ignored his critics until his behavior seemed bound to be the talk of all.", '', 'From this sad spectacle the senior nobles and privy gentlemen could only avert their eyes. Such things had led to disorder and ruin even in China, they said, and as discontent spread through the realm, the example of Yōkihi came more and more to mind, with many a painful consequence for the lady herself; yet she trusted i

In [27]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load model and tokenizer once, outside of the function
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

def summarize_chapter(chapter_text, max_length=150, min_length=40, length_penalty=2.0, num_beams=4):
    """
    Summarizes a given chapter text using BART model.
    """
    try:
        inputs = tokenizer([chapter_text], max_length=1024, return_tensors='pt', truncation=True)
        summary_ids = model.generate(inputs['input_ids'], max_length=max_length, min_length=min_length, length_penalty=length_penalty, num_beams=num_beams, early_stopping=True)
        return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    except Exception as e:
        # Handle exceptions such as too long texts or tokenization errors
        print(f"Error in summarization: {e}")
        return None



In [ ]:
summary = []
for chapter in chapters:
  whole_chapter = ''
  for each_sentence in chapter:
    whole_chapter = whole_chapter + each_sentence

  temp = summarize_chapter(whole_chapter)
  summary.append(temp)


summary[0]